<a href="https://colab.research.google.com/github/devBuzz142/inisw2-team7/blob/stt/whisoerx_raw2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip3 install --upgrade torch torchvision
#cuda 설정 위해 torchvision 설치

In [ ]:
!pip install setuptools-rust
#아래 코드 6줄은 cuda 설정 및 whisperx를 사용하기 위한 코드

In [ ]:
!pip install -U openai-whisper

In [ ]:
!pip install --upgrade --no-deps --force-reinstall git+https://github.com/openai/whisper.git

In [ ]:
!pip install git+https://github.com/m-bain/whisperx.git --upgrade

In [ ]:
!git clone https://github.com/m-bain/whisperX.git

In [ ]:
!sudo apt update && sudo apt install ffmpeg-python

In [ ]:
!pip install torch==2.0.0+cu117 torchvision==0.15.1+cu117 torchaudio==2.0.1 --index-url https://download.pytorch.org/whl/cu117

In [ ]:
import torch
import torchvision
import torch.nn
#cuda 사용하기 위해 import 

In [ ]:
torch.cuda.is_available()
#윗 코드 정상적으로 실행했다면 True 리턴

In [ ]:
%cd whisperX
#option

In [ ]:
import os
os.getcwd()
#혹시나해서 써둔 현재 작업 디렉토리 확인용 코드 

In [ ]:
import whisperx
import gc 
import ffmpeg
#미디어 파일들 사용하기 위한 라이브러리
import datetime
#SRT 파일 작성하기 위한 라이브러리


device = "cuda"  
#cuda 설정이 되어있지 않다면 "cpu"로 변경
audio_file = "conan_audio.mp3"
#작업 디렉토리 내에 오디오 파일을 넣어서 작업(사전에 !mkdir로 폴더 생성해도 작업 경로 옮긴 후, 경로입력해서 실행해도 좋다)
batch_size = 4 
# 높을수록 연산 속도가 더뎌지고 성능 높아짐
compute_type = "int8" 
# float으로 변경 시 성능 상승, 속도 감소


model = whisperx.load_model(r"medium", device, compute_type=compute_type)
#모델 사이즈를 지정할 수 있는 코드 (base medium large large-v1 large-v2)

audio = whisperx.load_audio(audio_file)
result = model.transcribe(audio, batch_size=batch_size)
#지정한 배치 크기로 오디오에서 음성을 전사(transcribe)하는 구간

model_a, metadata = whisperx.load_align_model(language_code=result["language"], device=device)
result = whisperx.align(result["segments"], model_a, metadata, audio, device, return_char_alignments=False)
# 결과값들과 GPU를 통해 처리한 metadata를 result 변수에 수합

#아래 코드블럭은 전부 srt파일을 작성하는 코드
def generate_srt_text(result):
    srt_text = ""
    segment_number = 1

    for segment in result["segments"]:
        start_time = segment["start"]
        end_time = segment["end"]
        text = segment["text"]

        # Convert time to srt format (hh:mm:ss,mmm)
        start_time_str = format_time_exact(start_time)
        end_time_str = format_time_exact(end_time)

        srt_text += f"{segment_number}\n{start_time_str} --> {end_time_str}\n{text}\n\n"
        segment_number += 1

    return srt_text

def format_time_exact(seconds):
    duration = datetime.timedelta(seconds=seconds)
    hours = duration.seconds // 3600
    minutes = (duration.seconds % 3600) // 60
    seconds = duration.seconds % 60
    milliseconds = duration.microseconds // 1000

    time_str = "{:02d}:{:02d}:{:02d},{:03d}".format(hours, minutes, seconds, milliseconds)
    return time_str


srt_text = generate_srt_text(result)

# SRT 파일이 작성된 후, 작업 디렉토리에 저장
with open("conan_sub.srt", "w", encoding="utf-8") as file:
    file.write(srt_text)